In [2]:
# Imports

import sys
import multiprocessing
import collections
import random
import warnings
import numpy as np
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import SimpleRNN, Dense, Bidirectional, LSTM, Dropout
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.compat.v1 import ConfigProto, InteractiveSession
from wandb.keras import WandbCallback
from data_repository import DataRepository
from sklearn.model_selection import StratifiedKFold

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#GPU
gpu_config = ConfigProto()
gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.3
gpu_config.gpu_options.allow_growth = True
session = InteractiveSession(config=gpu_config)

In [65]:
#load all

dirname = "/home/michi/signlang/artifacts/absolute3d:v2/"

repo = DataRepository(dirname)
num_classes=repo.numClasses
x, y = repo.getDataAndLabels()

['Bier', 'Computer', 'Deutschland', 'du', 'essen', 'Foto', 'Fußball', 'gestern', 'haben', 'Hallo', 'Haus', 'heute', 'Hose', 'Hunger', 'ich', 'Land', 'lernen', 'lieben', 'Mainz', 'morgen', 'rennen', 'Software', 'Sonne', 'Tag', 'trinken', 'Universität', 'unser', 'Welt', 'Wetter', 'zeigen']


In [70]:
#Config
batch_size=100
epochs=300
dropout = 0.2
node_size1 = 64
node_size2 = 256
node_size3 = 256
node_size4 = 256
node_size5 = 64
nodesizes  = [node_size2, node_size3, node_size4]
num_layers = 3
optimizer = "Adamax"

In [71]:
cp_path = "./models/model.hdf5"
CP = tf.keras.callbacks.ModelCheckpoint(
    cp_path,
    monitor="loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch",
)

In [73]:
# Model
model = Sequential()
model.add(Bidirectional(LSTM(node_size1, return_sequences=True), input_shape=(x.shape[1], x.shape[2]))
model.add(Dropout(rate=dropout))  

for i in range(0,num_layers):    #number of layers ramdom between 1 an 3
    model.add(Bidirectional(LSTM(nodesizes[i],return_sequences=True)))
    model.add(Dropout(rate=dropout))  

model.add(Bidirectional(LSTM(node_size5)))
model.add(Dropout(rate=dropout))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy',Precision(),Recall()])
model.summary()

model.fit(x,y,
epochs=epochs,
batch_size=batch_size,
shuffle=False,verbose=2, 
callbacks=[CP])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_33 (Bidirectio (1, 100, 128)             98816     
_________________________________________________________________
dropout_61 (Dropout)         (1, 100, 128)             0         
_________________________________________________________________
bidirectional_34 (Bidirectio (1, 100, 512)             788480    
_________________________________________________________________
dropout_62 (Dropout)         (1, 100, 512)             0         
_________________________________________________________________
bidirectional_35 (Bidirectio (1, 100, 512)             1574912   
_________________________________________________________________
dropout_63 (Dropout)         (1, 100, 512)             0         
_________________________________________________________________
bidirectional_36 (Bidirectio (1, 100, 512)           

InvalidArgumentError:  [_Derived_]  Invalid input_h shape: [1,1,64] [1,100,64]
	 [[{{node CudnnRNN}}]]
	 [[sequential_16/bidirectional_33/forward_lstm_62/StatefulPartitionedCall]]
	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_29]] [Op:__inference_train_function_1104845]

Function call stack:
train_function -> train_function -> train_function


In [45]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_13 (Bidirectio (None, 100, 512)          788480    
_________________________________________________________________
dropout_42 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 100, 512)          1574912   
_________________________________________________________________
dropout_43 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 128)               295424    
_________________________________________________________________
dropout_44 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 30)              

In [69]:
    #Test accuracy
    model_best_path = "/home/michi/ML/SignLanguageRecognition/lab/models/model.hdf5"
    best_model= load_model(filepath=model_best_path)

    #Convert to TFLite
    tflite_converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
    tflite_converter.experimental_new_converter = True
    tflite_model = tflite_converter.convert()
    open(os.path.join("/home/michi/ML/SignLanguageRecognition/lab/models/", "model.tflite"), "wb").write(tflite_model)

17641792